In [6]:
import torch
import torch.nn as nn
import dgl.function as fn
import torch.nn.functional as F
import dgl
import torch.optim as optim


class GCNLayer(nn.Module):
    def __init__(self,in_feats,out_feats,bias=True):
        super(GCNLayer,self).__init__()
        self.weight = nn.Parameter(torch.Tensor(in_feats,out_feats))
        if bias:
            self.bias = nn.Parameter(torch.zeros(out_feats))
        else:
            self.bias = None

        self.reset_parameter()
        
    def reset_parameter(self):
        nn.init.xavier_uniform_(self.weight)
    
    def forward(self,g,h):
        with g.local_scope():
            h = torch.matmul(h,self.weight)
            g.ndata['h'] = h * g.ndata['norm']
            g.update_all(message_func = fn.copy_u('h','m'),
                            reduce_func=fn.sum('m','h'))
            h = g.ndata['h']
            h = h * g.ndata['norm']
            if self.bias is not None:
                h = h + self.bias
            return h

In [7]:


class GCNModel(nn.Module):
    def __init__(self,in_feats,h_feats,num_classes,bias=True):
        super(GCNModel,self).__init__()
        self.conv1 = GCNLayer(in_feats,h_feats,bias)
        self.conv2 = GCNLayer(h_feats,num_classes,bias)
    
    def forward(self,g,in_feat):
        h = self.conv1(g,in_feat)
        h = F.relu(h)
        h = self.conv2(g,h)
        return h


In [8]:


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def train(g, model, optimizer, loss_fn, epochs):
    best_val_acc, best_test_acc = 0, 0
    # gain features and labels of datasets
    features, labels = g.ndata['feat'], g.ndata['label']
    train_mask, val_mask, test_mask = g.ndata['train_mask'], g.ndata[
        'val_mask'], g.ndata['test_mask']
    train_acc_list, val_acc_list, test_acc_list = [], [], []
    for e in range(epochs):
        logits = model(g, features)  # (N, label_nums)
        pred = logits.argmax(1)
        loss = loss_fn(logits[train_mask], labels[train_mask])
        # cal trian acc
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        # cal val acc
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        # cal test acc
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        train_acc_list.append(train_acc.item())
        val_acc_list.append(val_acc.item())
        test_acc_list.append(test_acc.item())

        # save result based on valid dataset
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (e + 1) % 5 == 0:
            print(
                'Epoch {}, loss: {:.3f}, train acc: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'
                .format(e + 1, loss, train_acc, val_acc, best_val_acc,
                        test_acc, best_test_acc))


if __name__ == "__main__":
    epochs = 200
    hidden_size = 32
    lr = 0.01
    weight_decay = 5e-4
    # download and loading dataset
    dataset = dgl.data.CoraGraphDataset(raw_dir="../temp/DGL/")
    g = dataset[0]
    # add self-loop
    g = dgl.remove_self_loop(g)
    g = dgl.add_self_loop(g)
    # gain degree matrix
    degs = g.out_degrees().float()
    # cal D^{-1/2}
    norm = torch.pow(degs, -0.5)
    norm[torch.isinf(norm)] = 0
    g.ndata['norm'] = norm.unsqueeze(1)

    model = GCNModel(in_feats=g.ndata['feat'].shape[1],
                     h_feats=hidden_size,
                     num_classes=dataset.num_classes)

    if torch.cuda.is_available():
        g = g.to(device)
        model = model.to(device)

    optimizer = optim.Adam(model.parameters(),
                           lr=lr,
                           weight_decay=weight_decay)
    loss_fn = nn.CrossEntropyLoss()
    train(g, model, optimizer, loss_fn, epochs)

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Epoch 5, loss: 1.897, train acc: 0.636, val acc: 0.334 (best 0.334), test acc: 0.352 (best 0.352)
Epoch 10, loss: 1.800, train acc: 0.821, val acc: 0.530 (best 0.530), test acc: 0.578 (best 0.578)
Epoch 15, loss: 1.672, train acc: 0.900, val acc: 0.592 (best 0.592), test acc: 0.610 (best 0.610)
Epoch 20, loss: 1.518, train acc: 0.943, val acc: 0.666 (best 0.666), test acc: 0.696 (best 0.696)
Epoch 25, loss: 1.348, train acc: 0.943, val acc: 0.722 (best 0.722), test acc: 0.739 (best 0.739)
Epoch 30, loss: 1.173, train acc: 0.950, val acc: 0.730 (best 0.730), test acc: 0.760 (best 0.760)
Epoch 35, loss: 1.003, train acc: 0.964, val acc: 0.764 (best 0.764), test acc: 0.784 (best 0.784)
Epoch 40, loss: 0.851, train acc: 0.971, val acc: 0.770 (best 0.772), test acc: 0.800 (best 0.797)
Epoch 45, loss: 0.722, trai